<a href="https://colab.research.google.com/github/Snazz2001/100-Days-Of-ML-Code/blob/master/qlora_zheng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch; torch.version.cuda

'12.6'

In [ ]:
!pip install "unsloth[cu121_ampere] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-kjvzde6y/unsloth_9829102c86fb426898cd4fb4693a4094
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-kjvzde6y/unsloth_9829102c86fb426898cd4fb4693a4094
  Resolved https://github.com/unslothai/unsloth.git to commit 8ea5338154859ed25b50366cb1264ed4d933eae3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 31.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 24.2 MB/s eta 0:00:00
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 512,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = 512,
)

Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 32 MLP layers.


In [ ]:
training_arguments = TrainingArguments(
        output_dir="./unsloth_results",
        eval_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=6,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=100,
        logging_steps=25,
        learning_rate=2e-4,
        eval_steps=50,
        optim='adamw_8bit',
        fp16=True, #change to fp16 if not using an Ampere GPU
        weight_decay=0.1,
        max_steps=500,
        warmup_ratio=0.01,
        lr_scheduler_type="linear",
)

In [ ]:
dataset = load_dataset("timdettmers/openassistant-guanaco")

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [ ]:
dataset['train']['text'][0]

'### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lead

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/9846 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/518 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
The model is already on multiple devices. Skipping the move to device specified in `args`.
Currently training with a batch size of: 4
The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, attention_mask. If text, attention_mask are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
skipped Embedding(32000, 4096, padding_idx=0): 125.0M params
bitsandbytes: will optimize Embedding(32000, 4096, padding_idx=0) in fp32
skipped: 125.0M params
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,846 | Num Epochs = 2 | Total steps = 500
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 6
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 6 x 1) = 24
 "-____-"     Trainable parameters = 28,31

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
Unsloth: Input IDs of shape torch.Size([4, 924]) with length 924 > the model's max sequence length of 512.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Unsloth: Will smartly offload gradients to save VRAM!


TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <function cross_entropy at 0x7c6b684de200>(*(), **{'input': GradTrackingTensor(lvl=1, value=
    FakeTensor(..., device='cuda:0', size=(s31, 32000))
), 'target': GradTrackingTensor(lvl=1, value=
    FakeTensor(..., device='cuda:0', size=(s7,), dtype=torch.int64)
), 'reduction': 'sum'}): got ValueError('Expected input batch_size (s31) to match target batch_size (s7).')

from user code:
   File "/usr/local/lib/python3.12/dist-packages/unsloth_zoo/fused_losses/cross_entropy_loss.py", line 281, in accumulate_chunk
    (chunk_loss, (unscaled_loss,)) = torch.func.grad_and_value(
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/apis.py", line 449, in wrapper
    return eager_transforms.grad_and_value_impl(
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/vmap.py", line 47, in fn
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/eager_tra

In [ ]:
!pip install "unsloth[cu121] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4d21c_nu/unsloth_2ea35744095d4020ac2ac9853d104a3e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4d21c_nu/unsloth_2ea35744095d4020ac2ac9853d104a3e
  Resolved https://github.com/unslothai/unsloth.git to commit 8ea5338154859ed25b50366cb1264ed4d933eae3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import (
    TrainingArguments,
)
from trl import SFTTrainer

model_name = "mistralai/Mistral-7B-v0.1"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 512,
    dtype = None,
    load_in_4bit = True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = 512,
)

training_arguments = TrainingArguments(
        output_dir="./unsloth_results_t4",
        eval_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=6,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=100,
        logging_steps=50,
        learning_rate=2e-4,
        eval_steps=50,
        optim='adamw_8bit',
        fp16=True, #change to fp16 if not using an Ampere GPU
        weight_decay=0.1,
        max_steps=500,
        warmup_ratio=0.01,
        lr_scheduler_type="linear",
        torch_compile=False,
)

dataset = load_dataset("timdettmers/openassistant-guanaco")

trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
        packing=False,
)

trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 32 MLP layers.
Repo card metadata block was not found. Setting CardData to empty.
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
The model is already on multiple devices. Skipping the move to device specified in `args`.
Currently training with a batch size of: 4
The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, attention_mask. If text, attention_mask are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this mess

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
Unsloth: Input IDs of shape torch.Size([4, 924]) with length 924 > the model's max sequence length of 512.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Unsloth: Will smartly offload gradients to save VRAM!


TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <function cross_entropy at 0x7b562f10d9e0>(*(), **{'input': GradTrackingTensor(lvl=1, value=
    FakeTensor(..., device='cuda:0', size=(s31, 32000))
), 'target': GradTrackingTensor(lvl=1, value=
    FakeTensor(..., device='cuda:0', size=(s7,), dtype=torch.int64)
), 'reduction': 'sum'}): got ValueError('Expected input batch_size (s31) to match target batch_size (s7).')

from user code:
   File "/usr/local/lib/python3.12/dist-packages/unsloth_zoo/fused_losses/cross_entropy_loss.py", line 281, in accumulate_chunk
    (chunk_loss, (unscaled_loss,)) = torch.func.grad_and_value(
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/apis.py", line 449, in wrapper
    return eager_transforms.grad_and_value_impl(
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/vmap.py", line 47, in fn
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/_functorch/eager_tra

In [ ]:
# 针对 Colab 的 T4 显卡优化的安装命令
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.29" "trl<0.9.0" peft accelerate bitsandbytes
!pip install unsloth_zoo

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-fq9dyb0j/unsloth_2f6d3d02b4624acc91197e69842338d2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-fq9dyb0j/unsloth_2f6d3d02b4624acc91197e69842338d2
  Resolved https://github.com/unslothai/unsloth.git to commit d8459b194c81a502962f16118509cdfa205483af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 14.5 MB/s eta 0:00:

In [10]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

from google.colab import drive
drive.mount('/content/drive')

# 1. 配置与模型加载
# 使用 Unsloth 预量化版本可以极大地减少之前的下载报错
model_name = "unsloth/mistral-7b-v0.3-bnb-4bit"
max_seq_length = 512

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None, # 自动检测
    load_in_4bit = True,
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # 训练必须是右填充
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = False # 消除警告

# 2. 设置 LoRA 参数
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    # 增加更多目标模块以提升 Mistral 的对话质量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # 使用优化的检查点
    random_state = 3407,
)

# 3. 设置训练参数
training_arguments = TrainingArguments(
    # output_dir="./unsloth_results_t4",
    output_dir="/content/drive/MyDrive/mistral_finetuning_checkpoints",
    eval_strategy="steps",        # 已更正为新版参数名
    do_eval=True,
    per_device_train_batch_size=2, # T4显存较小，建议设为2，靠累积步数补偿
    gradient_accumulation_steps=12, # 2 * 12 = 24 total batch size
    per_device_eval_batch_size=2,
    log_level="info",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    eval_steps=50,
    optim='adamw_8bit',           # 节省显存
    fp16=True,                    # T4 不支持 bf16，必须用 fp16
    bf16=False,
    weight_decay=0.01,
    max_steps=100,                # 演示用，可改回 500
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    torch_compile=False,          # 禁用编译以修复之前的 Dynamo 报错
    report_to="none",             # 避免跳出 wandb 登录框
)

# 4. 加载数据集
dataset = load_dataset("timdettmers/openassistant-guanaco")

# 5. 初始化训练器
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,                # 设为 False 配合 padding_side="right" 解决维度不匹配
)

# 6. 开始训练
trainer_stats = trainer.train()
# trainer.train(resume_from_checkpoint = True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--mistral-7b-v0.3-bnb-4bit/snapshots/cd23ad17d3fbbd4fce2ad35e61ae612e6052b4b5/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "bfloat16",
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 770,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modu

==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--mistral-7b-v0.3-bnb-4bit/snapshots/cd23ad17d3fbbd4fce2ad35e61ae612e6052b4b5/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "bfloat16",
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 770,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modu

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
FastLanguageModel.for_inference(model) # 开启推理模式
inputs = tokenizer(["### Human: 如何评价 Mistral 模型？\n### Assistant:"], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 64)
print(tokenizer.decode(outputs[0]))

NameError: name 'FastLanguageModel' is not defined

In [7]:
!ls /content/drive/MyDrive/mistral_finetuning_checkpoints